In [1]:
#!/usr/bin/env python3
import json
import re
from pathlib import Path
from datetime import datetime
import platform
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats  # for z-score
from pandas.tseries.holiday import USFederalHolidayCalendar
import scipy
# ----------------------------------------
# 1) Load the external category mapping
def load_category_mapping(path="categories.json"):
    with open(path, "r") as f:
        return json.load(f)

# ----------------------------------------
# 2) Read & parse the raw category-sales CSV
def read_sales_data(file_path):
    header_row = None
    with open(file_path, "r") as f:
        for i, line in enumerate(f):
            if re.match(r"^\s*Category", line):
                header_row = i
                break
    if header_row is None:
        raise ValueError(f"No 'Category' header found in {file_path}")

    df = pd.read_csv(
        file_path,
        header=header_row,
        engine="python",
        skipfooter=1
    )
    df.set_index(df.columns[0], inplace=True)
    df.drop("Total", errors="ignore", inplace=True)

    # strip $ and commas
    for col in df.columns:
        df[col] = (
            df[col]
              .astype(str)
              .str.replace(r"[\$,]", "", regex=True)
              .astype(float)
        )
    return df

# ----------------------------------------
# 3) Aggregate detailed → broad categories
def aggregate_sales_by_category(sales_df, category_mapping):
    df_long = (
        sales_df
          .reset_index()
          .melt(id_vars="Category", var_name="date", value_name="sales")
          .rename(columns={"Category":"category"})
    )
    df_long["date"] = pd.to_datetime(df_long["date"])
    df_long["broad"] = df_long["category"].map(category_mapping).fillna("Other")

    df_pivot = (
        df_long
          .pivot_table(
            index="date",
            columns="broad",
            values="sales",
            aggfunc="sum"
          )
          .fillna(0)
    )
    df_pivot["Total Sales"] = df_pivot.sum(axis=1)
    return df_pivot

# ----------------------------------------
# 4) Parse events into one-hot pivots
def parse_events(file_path):
    text = Path(file_path).read_text()
    matches = re.findall(r"\*\*(\d+/\d+/\d+)\*\* - .*?\[(.*?)\]", text)
    if not matches:
        return pd.DataFrame()
    rows = [
        {
          "date": pd.to_datetime(date_str, format="%m/%d/%Y"),
          "event": category
        }
        for date_str, category in matches
    ]
    df = (
        pd.DataFrame(rows)
          .assign(present=1)
          .pivot_table(
             index="date",
             columns="event",
             values="present",
             fill_value=0
          )
          .rename_axis(columns=None)
    )
    return df

# ----------------------------------------
# WEEK 5: Exploratory Data Analysis & Profiling
def profile_data(df, output_dir: Path):
    output_dir.mkdir(exist_ok=True, parents=True)
    print("\n=== WEEK 5: DATA PROFILE ===")
    print("Shape:", df.shape)
    print("Missing per column:\n", df.isna().sum())
    print("Duplicate dates? ", df.duplicated(subset=["date"]).any())
    print("\nDescriptive stats:\n", df.describe(include="all").T)

    # Time series plot of Total Sales
    if "Total Sales" in df.columns:
        plt.figure(figsize=(10,4))
        df.set_index("date")["Total Sales"].plot(title="Total Sales over Time")
        plt.ylabel("Sales ($)")
        plt.tight_layout()
        plt.savefig(output_dir / "ts_total_sales.png")
        plt.close()

        # Histogram
        plt.figure(figsize=(6,4))
        df["Total Sales"].hist(bins=30)
        plt.title("Distribution of Total Sales")
        plt.xlabel("Sales ($)")
        plt.tight_layout()
        plt.savefig(output_dir / "hist_total_sales.png")
        plt.close()

    # Correlation heatmap
    num = df.select_dtypes(include="number")
    corr = num.corr()
    plt.figure(figsize=(8,6))
    plt.matshow(corr, fignum=1)
    plt.title("Correlation Matrix", pad=20)
    plt.xticks(range(len(corr)), corr.columns, rotation=90)
    plt.yticks(range(len(corr)), corr.columns)
    plt.colorbar()
    plt.tight_layout()
    plt.savefig(output_dir / "corr_matrix.png")
    plt.close()

    # Save summary table
    summary = pd.DataFrame({
        "dtype": df.dtypes,
        "missing": df.isna().sum(),
        "unique": df.nunique(),
        "mean": df.mean(numeric_only=True),
        "std": df.std(numeric_only=True),
        "min": df.min(numeric_only=True),
        "max": df.max(numeric_only=True),
    })
    summary.to_csv(output_dir / "data_profile_summary.csv")

# ----------------------------------------
# WEEK 6: Missing-Value Handling
def handle_missing_values(df, output_dir: Path):
    output_dir.mkdir(exist_ok=True, parents=True)
    print("\n=== WEEK 6: MISSING-VALUE HANDLING ===")
    before = df.isna().sum()
    print("Missing before:\n", before)

    # Impute numeric columns with median
    medians = df.median(numeric_only=True)
    df_imputed = df.fillna(medians)

    after = df_imputed.isna().sum()
    print("Missing after:\n", after)

    # Log report
    report = pd.DataFrame({
        "missing_before": before,
        "missing_after": after,
        "filled": (before - after).clip(lower=0)
    })
    report.to_csv(output_dir / "missing_value_report.csv")
    return df_imputed

# ----------------------------------------
# WEEK 10: Syntactic & Pattern-Based Cleaning
def syntactic_cleaning(df):
    print("\n=== WEEK 10: SYNTACTIC CLEANING ===")
    if "product_category" in df.columns:
        df["product_category"] = (
            df["product_category"]
              .str.lower().str.strip()
              .str.replace(r"\s+", " ", regex=True)
        )
        print(" • Cleaned product_category whitespace & casing")
    else:
        print(" • No product_category column to clean")
    return df

# ----------------------------------------
def main():
    # reproducibility
    np.random.seed(0)
    random.seed(0)

    base_dir = Path.home() / "Downloads" / "IS 537"
    sales_files   = sorted(base_dir.glob("*category-sales*.csv"))
    events_files  = sorted(base_dir.glob("champaign-events-*.md"))
    weather_files = sorted(base_dir.glob("champaign_il_weather_*.csv"))

    # load mapping
    cat_map = load_category_mapping(base_dir / "categories.json")

    # ─── load & aggregate sales
    sales_list = [
        aggregate_sales_by_category(read_sales_data(fp), cat_map)
        for fp in sales_files
    ]
    sales_df = pd.concat(sales_list).groupby(level=0).mean()

    # ─── load & pivot events
    events_list = [parse_events(fp) for fp in events_files]
    events_df   = pd.concat(events_list).groupby(level=0).max()

    # ─── load weather
    weather_df = None
    if weather_files:
        wdfs = []
        for fp in weather_files:
            dfw = pd.read_csv(fp)
            dfw["date"] = pd.to_datetime(dfw["date"])
            dfw.set_index("date", inplace=True)
            wdfs.append(dfw.interpolate())
        weather_df = pd.concat(wdfs).groupby(level=0).mean()

    # ─── carve out date range, clipping future dates
    min_date   = min(sales_df.index.min(), events_df.index.min())
    today      = pd.Timestamp.today().normalize()
    max_source = max(sales_df.index.max(), events_df.index.max())
    max_date   = min(today, max_source)
    all_dates  = pd.date_range(start=min_date, end=max_date)

    # ─── build combined
    combined = pd.DataFrame(index=all_dates)
    combined = combined.join(sales_df).join(events_df).fillna(0)
    if weather_df is not None:
        combined = combined.join(weather_df)

    # reset index for profiling
    df0 = combined.reset_index().rename(columns={"index":"date"})

    # ───────────────────────────────────────────────────────────────
    # Week 5: Profiling
    profile_data(df0, base_dir / "profiling_outputs")

    # ───────────────────────────────────────────────────────────────
    # Week 6: Missing-Value Handling
    df1 = df0.set_index("date")
    df1 = handle_missing_values(df1, base_dir / "imputation_reports")
    df1 = df1.reset_index()

    # ───────────────────────────────────────────────────────────────
    # Week 7: Anomaly Detection (flag only)
    Q1  = df1["Total Sales"].quantile(0.25)
    Q3  = df1["Total Sales"].quantile(0.75)
    IQR = Q3 - Q1
    df1["is_outlier_iqr"] = ((df1["Total Sales"] < Q1 - 1.5*IQR) |
                             (df1["Total Sales"] > Q3 + 1.5*IQR))
    df1["zscore"]       = stats.zscore(df1["Total Sales"].fillna(0))
    df1["is_outlier_z"] = df1["zscore"].abs() > 3
    print(f"[Week 7] Flagged {df1['is_outlier_iqr'].sum()} IQR outliers and "
          f"{df1['is_outlier_z'].sum()} Z-score outliers")

    # ───────────────────────────────────────────────────────────────
    # Week 8: Duplicate Detection & Handling
    df1["is_duplicate_date"] = df1.duplicated(subset=["date"], keep=False)
    n_dup = df1["is_duplicate_date"].sum()
    print(f"[Week 8] Found {n_dup} rows in duplicate-date groups")
    pre = len(df1)
    df1 = df1.drop_duplicates(subset=["date"], keep="first")
    post = len(df1)
    print(f"[Week 8] Dropped {pre-post} duplicates; {post} rows remain")

    # ───────────────────────────────────────────────────────────────
    # Week 10: Syntactic Cleaning
    df1 = syntactic_cleaning(df1)

    # ───────────────────────────────────────────────────────────────
    # Week 11: Constraint Checks
    print("\n=== WEEK 11: CONSTRAINT VALIDATION ===")
    negs = df1[df1["Total Sales"] < 0]
    print(f" • Found {len(negs)} negative Total Sales entries")
    df1 = df1[df1["Total Sales"] >= 0]
    # sum-of-categories vs. total
    cat_cols = [c for c in df1.columns if c not in ["date","Total Sales","zscore",
                                                     "is_outlier_iqr","is_outlier_z",
                                                     "is_duplicate_date"]]
    mismatches = (df1[cat_cols].sum(axis=1).round(2) != df1["Total Sales"].round(2)).sum()
    print(f" • Found {mismatches} rows where category sums ≠ Total Sales")

    # Add holiday flag with pandas US calendar
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays(start=min_date, end=max_date)
    df1["is_holiday"] = df1["date"].isin(holidays)
    print(f" • Marked {df1['is_holiday'].sum()} federal holidays")

    # ───────────────────────────────────────────────────────────────
    # Week 12: Probabilistic Cleaning Signals
    print("\n=== WEEK 12: PROBABILISTIC SIGNALS ===")
    # combine signals into one error indicator
    df1["has_error_signal"] = (
        df1["is_outlier_iqr"] |
        df1["is_outlier_z"] |
        df1["is_duplicate_date"] |
        df1["is_holiday"]  # if you want to treat holiday days differently
    ).astype(int)
    print(f" • Computed probabilistic error signal for {df1['has_error_signal'].sum()} rows")

    # ───────────────────────────────────────────────────────────────
    # Week 13: Integration Sanity Checks
    print("\n=== WEEK 13: INTEGRATION CHECKS ===")
    # check for any dates missing weather or events columns
    missing_weather = df1["temperature"].isna().sum() if "temperature" in df1.columns else 0
    print(f" • Missing weather on {missing_weather} days")
    # ... add any other domain‐specific integration checks here

    # ───────────────────────────────────────────────────────────────
    # Week 14: Workflow Provenance & Environment Snapshot

# Week 14: Workflow Provenance & Environment Snapshot
    print("\n=== WEEK 14: PROVENANCE & ENVIRONMENT ===")
    env = {
        "python":   platform.python_version(),
        "pandas":   pd.__version__,
        "numpy":    np.__version__,
        "scipy":    scipy.__version__,
        "timestamp": datetime.utcnow().isoformat()
    }
    with open(base_dir / "env_snapshot.json", "w") as f:
        json.dump(env, f, indent=2)
    print(f"✓ Wrote environment snapshot to {base_dir/'env_snapshot.json'}")

    # ───────────────────────────────────────────────────────────────
    # Week 15: Reproducibility & Transparency
    print("\n=== WEEK 15: REPRODUCIBILITY ===")
    print(" • Random seeds set; code fully versioned in GitHub repo;")
    print("   env snapshot & profiling artifacts saved for full traceability")

    # ───────────────────────────────────────────────────────────────
    # Write out final
    out_path = base_dir / "combined_data_full.csv"
    df1.to_csv(out_path, index=False)
    print(f"\n✅ All done — combined data → {out_path}")

if __name__ == "__main__":
    main()



=== WEEK 5: DATA PROFILE ===
Shape: (497, 21)
Missing per column:
 date                     0
Drink (Coffee)           0
Drink (Non Coffee)       0
Food                     0
Other                    0
Retail                   0
Total Sales              0
Academic                 0
Campus                   0
Community                0
Holiday                  0
Popular                  0
Sports                   0
max_temp_celsius        33
min_temp_celsius        33
precipitation_mm        33
max_wind_speed_kmh      33
max_temp_fahrenheit     33
min_temp_fahrenheit     33
max_wind_speed_mph      33
precipitation_inches    33
dtype: int64
Duplicate dates?  False

Descriptive stats:
                       count unique        top freq      first       last  \
date                    497    497 2024-01-01    1 2024-01-01 2025-05-11   
Drink (Coffee)        497.0    NaN        NaT  NaN        NaT        NaT   
Drink (Non Coffee)    497.0    NaN        NaT  NaN        NaT        NaT   
Foo

/var/folders/_h/yxt5btd56fdfcnmx2q8hfr3m0000gn/T/ipykernel_8504/1545815962.py:112: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print("\nDescriptive stats:\n", df.describe(include="all").T)
/var/folders/_h/yxt5btd56fdfcnmx2q8hfr3m0000gn/T/ipykernel_8504/1545815962.py:141: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()



=== WEEK 6: MISSING-VALUE HANDLING ===
Missing before:
 Drink (Coffee)           0
Drink (Non Coffee)       0
Food                     0
Other                    0
Retail                   0
Total Sales              0
Academic                 0
Campus                   0
Community                0
Holiday                  0
Popular                  0
Sports                   0
max_temp_celsius        33
min_temp_celsius        33
precipitation_mm        33
max_wind_speed_kmh      33
max_temp_fahrenheit     33
min_temp_fahrenheit     33
max_wind_speed_mph      33
precipitation_inches    33
dtype: int64
Missing after:
 Drink (Coffee)          0
Drink (Non Coffee)      0
Food                    0
Other                   0
Retail                  0
Total Sales             0
Academic                0
Campus                  0
Community               0
Holiday                 0
Popular                 0
Sports                  0
max_temp_celsius        0
min_temp_celsius        0
precipitat